In [1]:
%load_ext autoreload
%autoreload
import os
import glob
import pandas as pd
import numpy as np
import subprocess as sp

Make subject directories

In [3]:
sub_list = ["101","102","103","104","106","107","108","109","111","112",
            "113","114","115","116","117","118","119","120","121","122",
            "123","124","125","126","127","129","130","131","133","134"]

Make scan and behav subdirectories within subject folders

In [10]:
subfolders = ["anat", "func", "fmap", "loc", "beh"]

In [11]:
for n in sub_list:
    sub_dir = "sub-%s/" % n
    for s in subfolders:
        os.makedirs(sub_dir + s)

Create minimal dataset description

In [2]:
import json

In [13]:
description = {"Name": "Imageability experiment",
               "BIDSVersion": "1.0.0-rc4",
               "Authors": ["Ami Tsuchida", "Brenda Milner"]}

In [14]:
with open("dataset_description.json", "w") as f:
    json.dump(description, f)

Create participants tsv

In [15]:
behav_dir = "/home/ami/Documents/Work/imaging/imageab/pilot2/behav"
summary = "Neuropsych.csv"

In [19]:
df = pd.read_csv(os.path.join(behav_dir, summary))
df.head()

,Subject ID,Age,Sex,Education,Area of Study,Handedness,F,A,S,Animal,...,Full IQ,Right (/30),Left(/30),L,R,Eyes Open,Eyes Closed,Average,Verbal,Visual
0,101,20,F,15,Cognitive Neuroscience,23,18,9,13,26,...,109,1,2,5,6,61,68,64.5,12,11
1,102,22,F,16,Psychology,49,14,14,14,19,...,92,2,1,11,13,58,69,63.5,17,13
2,103,19,F,13,Photography,23,8,9,14,19,...,88,0,0,2,2,69,62,65.5,1,11
3,104,28,M,20,Law,26,14,10,16,25,...,92,No data,No data,No data,No data,59,72,65.5,10,10
4,106,28,M,18,Biomedical Science,31,23,18,21,23,...,135,1,0,2,4,60,73,66.5,13,3


In [21]:
df.columns.values

array(['Subject ID', 'Age', 'Sex', 'Education', 'Area of Study',
       'Handedness', 'F', 'A', 'S', 'Animal', 'BD', 'SI', 'DS', 'MR', 'AR',
       'IN', 'CD', ' (SI+IN)*3/2', '(BD+MR)*3/2', 'DS+AR', 'CD*2',
       'all*10/7', 'VCI', 'PRI', 'WMI', 'PSI', 'Full IQ', 'Right (/30)',
       'Left(/30)', 'L', 'R', 'Eyes Open', 'Eyes Closed', 'Average',
       'Verbal', 'Visual'], dtype=object)

In [22]:
cols_to_keep = ['Subject ID', 'Age', 'Sex', 'Education', 'Area of Study',
                'Handedness', 'F', 'A', 'S', 'Animal', 'BD', 'SI', 'DS', 'MR', 'AR',
                'IN', 'CD', 'VCI', 'PRI', 'WMI', 'PSI', 'Full IQ']

In [27]:
col_names = ["subject", "age", "sex", "edu", "field", "hand",
             "fluency-f", "fluency-a", "fluency-s", "fluency-animal",
             "wais-bd", "wais-si", "wais-ds", "wais-mr", "wais-ar", 
             "wais-in", "wais-cd", "wais-vci", "wais-pri", "wais-wmi",
             "wais-psi", "wais-full"]

In [28]:
new_df = df[cols_to_keep]
new_df.columns = col_names
new_df.head()

,subject,age,sex,edu,field,hand,fluency-f,fluency-a,fluency-s,fluency-animal,...,wais-ds,wais-mr,wais-ar,wais-in,wais-cd,wais-vci,wais-pri,wais-wmi,wais-psi,wais-full
0,101,20,F,15,Cognitive Neuroscience,23,18,9,13,26,...,8,12,11,12,13,111,110,97,117,109
1,102,22,F,16,Psychology,49,14,14,14,19,...,10,8,8,13,8,115,80,94,88,92
2,103,19,F,13,Photography,23,8,9,14,19,...,5,10,8,11,12,103,81,79,112,88
3,104,28,M,20,Law,26,14,10,16,25,...,12,7,9,11,11,98,80,102,106,92
4,106,28,M,18,Biomedical Science,31,23,18,21,23,...,18,11,18,17,17,127,104,146,138,135


In [29]:
new_df.to_csv("participants.tsv", sep="\t", index=False)

In [31]:
participants_desc = {"sex": {"LongName": "Biological sex",
                             "Description": "Self-reported biological sex",
                             "Levels": {"F": "female", "M": "male"}},
                     "edu": {"LongName": "Educational level",
                             "Description": "Self-reported educational level",
                             "Units": "years"},
                     "field": {"LongName": "field of study",
                               "Description": "Self-reported area of study"},
                     "hand": {"LongName": "Handedness",
                              "Description": "In-house handedness questionnaire? Score ranges from 18-90, with lower value indicating strong R handedness"},
                     "fluency-f": {"LongName": "Verbal fluency-letter F",
                                   "Description": "# of words starting with the letter F that were generated in 1 minute"},
                     "fluency-a": {"LongName": "Verbal fluency-letter A",
                                   "Description": "# of words starting with the letter A that were generated in 1 minute"},
                     "fluency-s": {"LongName": "Verbal fluency-letter S",
                                   "Description": "# of words starting with the letter S that were generated in 1 minute"},
                     "fluency-animal": {"LongName": "Semantic verbal fluency-animal",
                                        "Description": "# of animals that were named in 1 minute"},
                     "wais-bd": {"LongName": "WAIS-IV Block Design",
                                "Description": "See WAIS-IV manual for description. Used for PRI calculation."},
                     "wais-si": {"LongName": "WAIS-IV Similarities",
                                 "Description": "See WAIS-IV manual for description. Used for VCI calculation."},
                     "wais-ds": {"LongName": "WAIS-IV Digit Span",
                                 "Description": "See WAIS-IV manual for description. Used for WMI calculation."},
                     "wais-mr": {"LongName": "WAIS-IV Matrix Reasoning",
                                 "Description": "See WAIS-IV manual for description. Used for PRI calculation."},
                     "wais-ar": {"LongName": "WAIS-IV Arithmetic",
                                 "Description": "See WAIS-IV manual for description. Used for WMI calculation."},
                     "wais-in": {"LongName": "WAIS-IV Information",
                                 "Description": "See WAIS-IV manual for description. Used for VCI calculation."},
                     "wais-cd": {"LongName": "WAIS-IV Coding",
                                 "Description": "See WAIS-IV manual for description. Used for PSI calculation."},
                     "wais-vci": {"LongName": "WAIS-IV Verbal Comprehension Index",
                                  "Description": "See WAIS-IV manual for description. Summary Index estimated from SI and IN in this study."},
                     "wais-pri": {"LongName": "WAIS-IV Perceptual Reasoning Index",
                                  "Description": "See WAIS-IV manual for description. Summary Index estimated from BD and MR in this study."},
                     "wais-wmi": {"LongName": "WAIS-IV Working Memory Index",
                                  "Description": "See WAIS-IV manual for description. Summary Index estimated from DS and AR in this study."},
                     "wais-psi": {"LongName": "WAIS-IV Processing Speed Index",
                                  "Description": "See WAIS-IV manual for description. Summary Index estimated from CD in this study."},
                     "wais-full": {"LongName": "WAIS-IV full IQ",
                                   "Description": "See WAIS-IV manual for description. Estimated full IQ from wais sub-tests used in the study."}}

In [32]:
with open("participants.json", "w") as f:
    json.dump(participants_desc, f)

Make code directory to store codes

In [4]:
os.makedirs("code")

In [10]:
with open("code/__init__.py", "w"):
    pass

Make ds001 directory in RAW nifti dir and create scan_key.json

In [2]:
imaging_dir = "/home/ami/Documents/Work/imaging"
RAWnifti_dir = os.path.join(imaging_dir, "RAW_All", "nifti")


In [ ]:
os.makedirs(os.path.join(RAWnifti_dir, "ds001"))

In [3]:
scan_key = {"anat": {"protocol": "MPRAGEADNI",
                     "timepoints": 1,
                     "raw_nifti_name": ["sub-(sub)_(prot)(series)"],
                     "bids_type": "anat",
                     "bids_name": ["sub-(sub)_T1w"]},
            "func-rest": {"protocol": "BOLDMOSAIC64_Resting",
                          "timepoints": 143,
                          "raw_nifti_name": ["sub-(sub)_(prot)(series)"],
                          "bids_type": "func",
                          "bids_name": ["sub-(sub)_task-rest_(run)_bold"]},
            "func-task": {"protocol": "BOLDMOSAIC64_Task",
                          "timepoints": 610,
                          "raw_nifti_name": ["sub-(sub)_(prot)(series)"],
                          "bids_type": "func",
                          "bids_name": ["sub-(sub)_task-imag_bold"]},
            "func-task1": {"protocol": "BOLDMOSAIC64_Task_1",
                          "timepoints": 305,
                          "raw_nifti_name": ["sub-(sub)_(prot)(series)"],
                          "bids_type": "func",
                          "bids_name": ["sub-(sub)_task-imag_run-01_bold"]},
            "func-task2": {"protocol": "BOLDMOSAIC64_Task_2",
                          "timepoints": 305,
                          "raw_nifti_name": ["sub-(sub)_(prot)(series)"],
                          "bids_type": "func",
                          "bids_name": ["sub-(sub)_task-imag_run-02_bold"]},
            "fmap-mag": {"protocol": "gre_field_mapping",
                         "timepoints": 2,
                         "raw_nifti_name": ["sub-(sub)_(prot)(series)",
                                            "sub-(sub)_(prot)(series)_e2"],
                         "bids_type": "fmap",
                         "bids_name": ["sub-(sub)_(run)_magnitude1",
                                       "sub-(sub)_(run)_magnitude2"]},
            "fmap-phase": {"protocol": "gre_field_mapping",
                           "timepoints": 1,
                           "raw_nifti_name": ["sub-(sub)_(prot)(series)_e2"],
                           "bids_type": "fmap",
                           "bids_name": ["sub-(sub)_(run)_phasediff"]},
            }

In [4]:
from code.doit_bids_setup import get_nifti_names

In [18]:
scaninfo_txt = "%s/ds001/sub-101/scaninfo.txt" % RAWnifti_dir
scaninfo = pd.read_csv(scaninfo_txt, header=None, sep='\s*')
scaninfo.head()

/home/ami/Documents/Work/imaging/ds001/ds001env/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,0,1,2,3,4,5,6,7
0,1,localizer-32ch,ok,512,512,5,1,91795557
1,2,SAGTRUFISP,ok,256,256,23,1,91795964
2,3,BOLDMOSAIC64_Resting,ok,64,64,42,143,91795980
3,4,BOLDMOSAIC64_Resting,ok,64,64,42,143,91795030
4,5,gre_field_mapping,ok,64,64,42,2,91790826


In [5]:
names = get_nifti_names("101", scan_key)
names

code/doit_bids_setup.py:51: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  scaninfo = pd.read_csv(scaninfo_txt, header=None, sep='\s*')


{'sub-101_BOLDMOSAIC64_Resting10': 'sub-101_task-rest_run-04_bold',
 'sub-101_BOLDMOSAIC64_Resting3': 'sub-101_task-rest_run-01_bold',
 'sub-101_BOLDMOSAIC64_Resting4': 'sub-101_task-rest_run-02_bold',
 'sub-101_BOLDMOSAIC64_Resting9': 'sub-101_task-rest_run-03_bold',
 'sub-101_BOLDMOSAIC64_Task8': 'sub-101_task-imag_bold',
 'sub-101_MPRAGEADNI7': 'sub-101_T1w',
 'sub-101_gre_field_mapping11': 'sub-101_run-02_magnitude1',
 'sub-101_gre_field_mapping11_e2': 'sub-101_run-02_magnitude2',
 'sub-101_gre_field_mapping12_e2': 'sub-101_run-02_phasediff',
 'sub-101_gre_field_mapping5': 'sub-101_run-01_magnitude1',
 'sub-101_gre_field_mapping5_e2': 'sub-101_run-01_magnitude2',
 'sub-101_gre_field_mapping6_e2': 'sub-101_run-01_phasediff'}

In [8]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [10]:
replace_all("hello world!", {"hello":"byebye", "world":"earth", "hi": "salut"})

'byebye earth!'

In [5]:
doit_bids_setup.doit(['list', '--all'])

dicom2nifti         
dicom2nifti:101     Use dcm2niix to convert dicom to nifti
dicom2nifti:102     Use dcm2niix to convert dicom to nifti
dicom2nifti:103     Use dcm2niix to convert dicom to nifti
dicom2nifti:104     Use dcm2niix to convert dicom to nifti
dicom2nifti:106     Use dcm2niix to convert dicom to nifti
dicom2nifti:107     Use dcm2niix to convert dicom to nifti
dicom2nifti:108     Use dcm2niix to convert dicom to nifti
dicom2nifti:109     Use dcm2niix to convert dicom to nifti
dicom2nifti:111     Use dcm2niix to convert dicom to nifti
dicom2nifti:112     Use dcm2niix to convert dicom to nifti
dicom2nifti:113     Use dcm2niix to convert dicom to nifti
dicom2nifti:114     Use dcm2niix to convert dicom to nifti
dicom2nifti:115     Use dcm2niix to convert dicom to nifti
dicom2nifti:116     Use dcm2niix to convert dicom to nifti
dicom2nifti:117     Use dcm2niix to convert dicom to nifti
dicom2nifti:118     Use dcm2niix to convert dicom to nifti
dicom2nifti:119     Use dcm2niix to

0

In [8]:
test = "dcm2niix -b y -z y -f %s%%p%%s -o '%s' %s" % ("first","second", "third")
print test


dcm2niix -b y -z y -f first%p%s -o 'second' third


In [15]:
study_dir = "%s/ds001/" % imaging_dir
participants_info = "%s/participants.tsv" % study_dir
participants_df = pd.read_csv(participants_info, sep="\t")
participants = participants_df.subject
participants

0     101
1     102
2     103
3     104
4     106
5     107
6     108
7     109
8     111
9     112
10    113
11    114
12    115
13    116
14    117
15    118
16    119
17    120
18    121
19    122
20    123
21    124
22    125
23    126
24    127
25    129
26    130
27    131
28    133
29    134
Name: subject, dtype: int64

In [29]:
for subj in participants:
    src = glob.glob("%s%s*" % (RAW_dir, subj))[0]
    print src


/home/ami/Documents/Work/imaging/RAW_All/dicom/101_20140110
/home/ami/Documents/Work/imaging/RAW_All/dicom/102_20140113
/home/ami/Documents/Work/imaging/RAW_All/dicom/103_20140108
/home/ami/Documents/Work/imaging/RAW_All/dicom/104_20131218
/home/ami/Documents/Work/imaging/RAW_All/dicom/106_20140108
/home/ami/Documents/Work/imaging/RAW_All/dicom/107_20140109
/home/ami/Documents/Work/imaging/RAW_All/dicom/108_20140206
/home/ami/Documents/Work/imaging/RAW_All/dicom/109_20140207
/home/ami/Documents/Work/imaging/RAW_All/dicom/111_20140321
/home/ami/Documents/Work/imaging/RAW_All/dicom/112_20140324
/home/ami/Documents/Work/imaging/RAW_All/dicom/113_20140314
/home/ami/Documents/Work/imaging/RAW_All/dicom/114_20140319
/home/ami/Documents/Work/imaging/RAW_All/dicom/115_20140324
/home/ami/Documents/Work/imaging/RAW_All/dicom/116_20140321
/home/ami/Documents/Work/imaging/RAW_All/dicom/117_20140320
/home/ami/Documents/Work/imaging/RAW_All/dicom/118_20140327
/home/ami/Documents/Work/imaging/RAW_All